### Imports

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')

In [2]:
from ECD import BatchOptimizer as BO_ECD
from DECD_Old_Fixed_Mult import FixedBatchOptimizer as BO_DECD


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [3]:
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

### Single ECD g0 ->g1

Initial Testing Code

In [4]:
#The target oscillator state.
N =10
psi_i = qt.basis(N, 0) # initial state
psi_t = qt.basis(N,1) #target state

#Opt Params
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
ecd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 1000, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.9999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [qt.tensor(qt.basis(2,0),psi_i)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_t)], #end in |e> |target>.
'name' : 'Fock %d' % 1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [7]:
#Optimizer 
ecd_opt = BO_ECD(**ecd_opt_params)
ecd_opt.print_info()

optimization_type: state transfer
N_multistart: 10
N_blocks: 5
term_fid: 0.9999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 1000
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 10
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-0.99802+0.47732j  0.54524-0.50095j -0.26729-0.38053j  1.97363+2.03726j
 -0.45516-0.17712j]
alphas:        [0.+0.j]
phis (deg):    [   0.       -45.82933 -162.24318   -4.07417   62.59441]
thetas (deg):  [ -74.94595 -164.74908  -88.97781  111.07859  155.36848]
Max Fidelity:  0.177055




In [6]:
ecd_opt.optimize()

Start time: 2022-05-20 09:35:23
 Epoch: 16 / 1000 Max Fid: 0.984004 Avg Fid: 0.809492 Max dFid: 0.036194 Avg dFid: 0.009187 Elapsed time: 0:00:05.121724 Remaing time: 0:05:14.986049

KeyboardInterrupt: 

In [8]:
ecd_opt.best_circuit()

{'fidelity': 0.17705488,
 'betas': array([-0.99802047+0.47732067j,  0.54524374-0.50095284j,
        -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
        -0.45516223-0.17711596j], dtype=complex64),
 'alphas': array([0.+0.j], dtype=complex64),
 'phis': array([ 0.        , -0.79987264, -2.8316777 , -0.07110763,  1.0924785 ],
       dtype=float32),
 'thetas': array([-1.3080535, -2.8754141, -1.5529559,  1.9386871,  2.7116916],
       dtype=float32)}

### Feeding ECD params to DECD

#### Convert ECD angles to DECD angles
Add gamma

In [9]:
#obtain ECD params
circ = ecd_opt.best_circuit()
betas = circ['betas']
alphas = circ['alphas']
phis = circ['phis']
thetas = circ['thetas']

#gammas (displacement of second mode)
gammas = np.copy(betas)
for i in range(len(gammas)):
    gammas[i] = 0.0 +0.0j

#etas (just ignore this)
etas = np.copy(betas)
for i in range(len(etas)):
    etas[i] = np.pi/2 #pi/2

#new initial params (Needs 2 alphas for mode 1 and mode 2)
init_params = [betas, gammas, alphas, alphas, phis, etas, thetas]

In [10]:
etas

array([1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j,
       1.5707964+0.j], dtype=complex64)

In [11]:
#Since DECD has more than 1 multistart, we will make sure all multistarts
# have the same initial params-- namely the ecd onces
N_multistart = 10
init_params_more_multi = [np.array([i for j in range(N_multistart)]) for i in init_params]

In [12]:
init_params_more_multi

[array([[-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
         -0.45516223-0.17711596j],
        [-0.99802047+0.47732067j,  0.54524374-0.50095284j,
         -0.2672854 -0.

#### Initial Testing Code

In [13]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [14]:
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

Quantum object: dims = [[10, 10], [1, 1]], shape = (100, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [15]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
decd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.1, #adam learning rate
'term_fid' : 0.99, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



#### Running DECD

In [16]:
decd_opt = BO_DECD(**decd_opt_params)
tf.config.run_functions_eagerly(True) # for some reason, important
decd_opt.print_info()

[array([-0.99802047+0.47732067j,  0.54524374-0.50095284j,
       -0.2672854 -0.38052833j,  1.9736321 +2.0372593j ,
       -0.45516223-0.17711596j], dtype=complex64), array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64), array([0.+0.j], dtype=complex64), array([0.+0.j], dtype=complex64), array([ 0.        , -0.79987264, -2.8316777 , -0.07110763,  1.0924785 ],
      dtype=float32), array([1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j, 1.5707964+0.j,
       1.5707964+0.j], dtype=complex64), array([-1.3080535, -2.8754141, -1.5529559,  1.9386871,  2.7116916],
      dtype=float32)]
Fixed construct needed matrices called
optimization_type: state transfer
N_multistart: 1
N_blocks: 5
term_fid: 0.99
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.1
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: [array([-0.99

C:\Users\Eesh Gupta\anaconda\lib\site-packages\tensorflow\python\framework\constant_op.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


In [19]:
decd_opt.optimize()

Start time: 2022-05-20 09:03:32
 Epoch: 1 / 100 Max Fid: 0.996351 Avg Fid: 0.996351 Max dFid: -0.002094 Avg dFid: -0.002094 Elapsed time: 0:00:03.399668 Remaing time: 0:05:36.567153

 Optimization stopped. Term fidelity reached.

optimization_type: state transfer
N_multistart: 1
N_blocks: 5
term_fid: 0.99
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.1
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: [array([ 1.18335+0.58206j, -0.4436 +0.62623j,  0.23375+0.67681j,
       -0.48112+0.07249j,  0.00008+0.00007j], dtype=complex64), array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex64), array([0.+0.j], dtype=complex64), array([0.+0.j], dtype=complex64), array([ 0.     , -1.57089,  1.57069, -1.57089,  3.14152], dtype=float32), array([1.5708+0.j, 1.5708+0.j, 1.5708+0.j, 1.5708+0.j, 1.5708+0.j],
      dtyp

'2022-05-20 09:03:32'

In [35]:
decd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.gammas_rho,
                decd_opt.gammas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.alphas2_rho,
                decd_opt.alphas2_angle,
                decd_opt.phis,
                decd_opt.etas,
                decd_opt.thetas,)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0747334e-13>

#### Inspecting Gradients near minima

In [74]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / decd_opt.parameters["N_multistart"]
    return avg_loss

In [75]:
variables = [decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.phis,
            decd_opt.thetas]

with tf.GradientTape() as tape:
    fid = decd_opt.batch_fidelities(decd_opt.betas_rho,
            decd_opt.betas_angle,
            decd_opt.gammas_rho,
            decd_opt.gammas_angle,
            decd_opt.alphas1_rho,
            decd_opt.alphas1_angle,
            decd_opt.alphas2_rho,
            decd_opt.alphas2_angle,
            decd_opt.phis,
            decd_opt.etas,
            decd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

[None, None, None, None, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.00117935],
       [-0.02273876],
       [-0.00517935],
       [ 0.00023586],
       [ 0.00974743]], dtype=float32)>, <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.01013791],
       [-0.02023907],
       [-0.02267454],
       [-0.02169513],
       [ 0.00093247]], dtype=float32)>]


In [70]:
fid

<tf.Tensor: shape=(), dtype=float32, numpy=0.9899562>

In [65]:
@tf.function
def loss_fun(fids):
    #print('loss fun called')
    # I think it's important that the log is taken before the avg
    losses = tf.math.log(1 - fids)
    avg_loss = tf.reduce_sum(losses) / ecd_opt.parameters["N_multistart"]
    return avg_loss

In [66]:
variables = [ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.phis,
            ecd_opt.thetas]

with tf.GradientTape() as tape:
    fid = ecd_opt.batch_fidelities(ecd_opt.betas_rho,
            ecd_opt.betas_angle,
            ecd_opt.alphas_rho,
            ecd_opt.alphas_angle,
            ecd_opt.phis,
            ecd_opt.thetas,)
    new_loss = loss_fun(fid)
    #tape = tf.GradientTape()
    dloss_dvar = tape.gradient(new_loss, variables)
    print(dloss_dvar)
                   

[<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[ 3.5368568e-05,  1.4582398e-06, -1.6648934e-04,  5.4512825e-04,
        -1.3194750e-08, -6.6370852e-03,  6.2279825e-05,  1.4816676e-05,
         4.1190069e-05,  2.0490918e-06],
       [-1.5194964e-04,  1.5983483e-07, -1.3974009e-03, -3.0532166e-02,
        -7.2742793e-08,  2.5009643e-02, -2.6463054e-04,  1.6805268e-04,
        -1.3527540e-02,  4.4460765e-05],
       [ 2.1258564e-04, -1.5518254e-06,  1.1042421e-03,  4.2838037e-02,
         2.4293021e-07, -2.7163293e-02,  1.2391097e-04, -1.1924920e-03,
         1.8107558e-02,  6.8114027e-06],
       [ 2.9340164e-05, -7.1960562e-06, -6.4653985e-04,  1.6693925e-02,
        -5.0462859e-07,  6.7628153e-02, -3.2525433e-05, -1.2029792e-04,
        -3.5097595e-02,  1.3094203e-05],
       [ 4.7652480e-05,  1.4139036e-06, -5.0548260e-04, -4.2997161e-03,
         4.1585000e-07,  4.2433850e-03,  6.5994158e-05, -3.6126067e-04,
         3.3981081e-02,  7.1213844e-06]], dtype=float32)>, <tf.Ten

In [67]:
fid

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.9150901 , 0.8617574 , 0.8677291 , 0.91820776, 0.8480251 ,
       0.98889935, 0.99238324, 0.9129576 , 0.9898716 , 0.8917448 ],
      dtype=float32)>